# IMPORTS AND PATHS

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import re

In [2]:
# PATHS

DATA_PATH = r"D:\Users\orosh\Documents\Research\Datasets\eicu-2.0\eicu_crd"
OUTPUT_PATH = r"D:\Users\orosh\Documents\Research\Datasets\eicu-2.0\processed"

DIAGNOSIS_DATA_PATH = "diagnosis.csv.gz"
VITALS_DATA_PATH = "vitalPeriodic.csv.gz"
PATIENT_DATA_PATH = "patient.csv.gz"

In [3]:
# PARAMETERS

SCENARIO = "core"  # core, slim, ultra_minimal

# INSPECT TABLES

In [10]:
# Load headers of each CSV and files the columns names and shapes
def load_csv_headers(data_path, filename):
    filepath = os.path.join(data_path, filename)
    df = pd.read_csv(filepath, nrows=0)
    return df.columns.tolist(), df.shape[1]

# Load the headers for each CSV file
diagnosis_headers, diagnosis_shape = load_csv_headers(DATA_PATH, DIAGNOSIS_DATA_PATH)
vitals_headers, vitals_shape = load_csv_headers(DATA_PATH, VITALS_DATA_PATH)
patient_headers, patient_shape = load_csv_headers(DATA_PATH, PATIENT_DATA_PATH)

# Print the headers and shapes
print("Diagnosis Headers:", diagnosis_headers)
print("Diagnosis Shape:", diagnosis_shape)
print("Vitals Headers:", vitals_headers)
print("Vitals Shape:", vitals_shape)
print("Patient Headers:", patient_headers)
print("Patient Shape:", patient_shape)

# Diagnosis Headers: ['diagnosisid', 'patientunitstayid', 'activeupondischarge', 'diagnosisoffset', 'diagnosisstring', 'icd9code', 'diagnosispriority']
# Diagnosis Shape: 7
# Vitals Headers: ['vitalperiodicid', 'patientunitstayid', 'observationoffset', 'temperature', 'sao2', 'heartrate', 'respiration', 'cvp', 'etco2', 'systemicsystolic', 'systemicdiastolic', 'systemicmean', 'pasystolic', 'padiastolic', 'pamean', 'st1', 'st2', 'st3', 'icp']
# Vitals Shape: 19
# Patient Headers: ['patientunitstayid', 'patienthealthsystemstayid', 'gender', 'age', 'ethnicity', 'hospitalid', 'wardid', 'apacheadmissiondx', 'admissionheight', 'hospitaladmittime24', 'hospitaladmitoffset', 'hospitaladmitsource', 'hospitaldischargeyear', 'hospitaldischargetime24', 'hospitaldischargeoffset', 'hospitaldischargelocation', 'hospitaldischargestatus', 'unittype', 'unitadmittime24', 'unitadmitsource', 'unitvisitnumber', 'unitstaytype', 'admissionweight', 'dischargeweight', 'unitdischargetime24', 'unitdischargeoffset', 'unitdischargelocation', 'unitdischargestatus', 'uniquepid']
# Patient Shape: 29

Diagnosis Headers: ['diagnosisid', 'patientunitstayid', 'activeupondischarge', 'diagnosisoffset', 'diagnosisstring', 'icd9code', 'diagnosispriority']
Diagnosis Shape: 7
Vitals Headers: ['vitalperiodicid', 'patientunitstayid', 'observationoffset', 'temperature', 'sao2', 'heartrate', 'respiration', 'cvp', 'etco2', 'systemicsystolic', 'systemicdiastolic', 'systemicmean', 'pasystolic', 'padiastolic', 'pamean', 'st1', 'st2', 'st3', 'icp']
Vitals Shape: 19
Patient Headers: ['patientunitstayid', 'patienthealthsystemstayid', 'gender', 'age', 'ethnicity', 'hospitalid', 'wardid', 'apacheadmissiondx', 'admissionheight', 'hospitaladmittime24', 'hospitaladmitoffset', 'hospitaladmitsource', 'hospitaldischargeyear', 'hospitaldischargetime24', 'hospitaldischargeoffset', 'hospitaldischargelocation', 'hospitaldischargestatus', 'unittype', 'unitadmittime24', 'unitadmitsource', 'unitvisitnumber', 'unitstaytype', 'admissionweight', 'dischargeweight', 'unitdischargetime24', 'unitdischargeoffset', 'unitdisch

In [11]:
# Print example how what is inside "vitalperiodicid" col of vitalPeriodic.csv
vitals_path = os.path.join(DATA_PATH, VITALS_DATA_PATH)
vitals_df = pd.read_csv(vitals_path, nrows=50)
print("Example of vitalperiodicid column:")
print(vitals_df["vitalperiodicid"].head(10))

Example of vitalperiodicid column:
0    37376747
1    37404957
2    37385871
3    37401664
4    37377404
5    37394164
6    37374753
7    37438070
8    37425591
9    37393215
Name: vitalperiodicid, dtype: int64


# BUILD DATASET

## Build a stay-level infection-evidence table

| Tier                                | Tag value | Definition (AND/OR logic)                                                                                                                                                     | Typical PPV / Sensitivity\*                                           | Keep / Drop |
| ----------------------------------- | --------- | ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------- | ----------- |
| **Tier 5 – Definite infection**     | **5**     | (a) Explicit sepsis/septic-shock ICD-9 **OR** 038.\* septicemia **AND** (b) positive blood/sterile culture within ±24 h of the code **AND** (c) ≥ 4 d of systemic antibiotics | PPV ≈ 0.95, Sens ≈ 0.55([PMC][1], [physionet.org][2])                 | **Keep**    |
| **Tier 4 – Probable infection**     | 4         | Any one of: explicit sepsis code **OR** positive culture **OR** ≥ 2 d IV/PO antibiotics **AND** ICD-9 from infectious-disease chapter 001–139                                 | PPV ≈ 0.85, Sens ≈ 0.75([PMC][3], [PMC][4])                           | maybe       |
| **Tier 3 – Possible infection**     | 3         | Antibiotics ≥ 2 d **OR** infectious ICD-9 alone **OR** positive culture alone                                                                                                 | Balanced set; noisy                                                   | maybe       |
| **Tier 2 – Probably non-infected**  | 2         | ≤ 1 d antibiotic exposure **AND** no infection codes **AND** all cultures negative/no cultures                                                                                | PPV ≈ 0.25, Sens ≈ 0.90 (non-inf.)([jamanetwork.com][5])              | maybe       |
| **Tier 1 – Definite non-infection** | **1**     | **No** infection ICD-9 **AND** **no** antibiotics **AND** no cultures drawn                                                                                                   | Specificity > 0.95 for “clean” stays (e.g., elective CABG) ([PMC][6]) | **Keep**    |

[1]: https://pmc.ncbi.nlm.nih.gov/articles/PMC4403835/?utm_source=chatgpt.com "Validity of administrative data in recording sepsis: a systematic review"
[2]: https://physionet.org/content/eicu-crd-demo/2.0.1/microLab.csv.gz?utm_source=chatgpt.com "microLab.csv.gz - eICU Collaborative Research Database Demo"
[3]: https://pmc.ncbi.nlm.nih.gov/articles/PMC8217098/?utm_source=chatgpt.com "Determining the Electronic Signature of Infection in Electronic Health ..."
[4]: https://pmc.ncbi.nlm.nih.gov/articles/PMC3568444/?utm_source=chatgpt.com "Identifying Patients with Severe Sepsis Using Administrative Claims"
[5]: https://jamanetwork.com/journals/jamanetworkopen/fullarticle/2768537?utm_source=chatgpt.com "Assessment of ICD-9 Diagnosis Codes to Identify Pneumonia ..."
[6]: https://pmc.ncbi.nlm.nih.gov/articles/PMC6132188/?utm_source=chatgpt.com "The eICU Collaborative Research Database, a freely available multi ..."


In [12]:
# ---------- 1-A  Diagnosis evidence ---------------------------------------
dx_cols = ["patientunitstayid", "diagnosisoffset",
           "icd9code", "diagnosisstring"]
dx = pd.read_csv(os.path.join(DATA_PATH, "diagnosis.csv.gz"),
                 usecols=dx_cols, low_memory=False)

INFECT_PREFIX = [f"{i:03d}" for i in range(1,140)]                # 001-139
EXPLICIT = {"99591", "99592", "78552", "038"}                     # sepsis / septicemia
txt_re  = re.compile(r"(sepsis|infection|pneumonia|bacteremia|cellulitis)",
                     re.I)

dx["infect_code"] = dx["icd9code"].astype(str).str[:3].isin(INFECT_PREFIX)
dx["explicit"]    = dx["icd9code"].astype(str).str.replace(".", "").isin(EXPLICIT)
dx["infect_txt"]  = dx["diagnosisstring"].fillna("").str.contains(txt_re)

onset_dx = (dx[dx["infect_code"] | dx["infect_txt"]]
            .groupby("patientunitstayid")["diagnosisoffset"]
            .min()
            .rename("onset_dx"))

# ---------- 1-B  Culture evidence -----------------------------------------
lab_cols = ["patientunitstayid", "culturetakenoffset", "organism"]
lab = pd.read_csv(os.path.join(DATA_PATH, "microLab.csv.gz"),
                  usecols=lab_cols, low_memory=False)

lab["pos_cx"] = lab["organism"].fillna("").str.strip().str.lower().ne("") & \
                lab["organism"].str.contains(r"no growth", case=False, na=False).eq(False)

onset_cx = (lab[lab["pos_cx"]]
            .groupby("patientunitstayid")["culturetakenoffset"]
            .min()
            .rename("onset_cx"))

# ---------- 1-C  Antibiotic evidence --------------------------------------
abx_pat = re.compile(
    r"(cef|ceph|penem|piperacillin|tazo|penicillin|vanco|linezolid|"
    r"azithro|levo|cipro|clinda|flagyl|meropenem|imipenem|unasyn|zosyn)",
    re.I)

med_cols = ["patientunitstayid", "drugstartoffset",'drugstopoffset',
            "drugname", "drugordercancelled"]
med = pd.read_csv(os.path.join(DATA_PATH, "medication.csv.gz"),
                  usecols=med_cols, low_memory=False)

med["is_abx"] = med["drugname"].fillna("").str.contains(abx_pat)
med = med[(med["is_abx"]) & (med["drugordercancelled"].fillna("No").str.lower() != "yes")]

# treat missing stop-offset as 24 h duration or ICU discharge
med['drugstopoffset'] = med['drugstopoffset'].fillna(med['drugstartoffset'] + 1440)
med['dot_minutes']    = med['drugstopoffset'] - med['drugstartoffset']
# abx_days   = (med.groupby("patientunitstayid").size()
#                       .div(48)                             # rough 30-min-bin → day
#                       .rename("abx_days"))
abx_days = (med.groupby('patientunitstayid')['dot_minutes']
                  .sum()
                  .div(1440)          # minutes → days
                  .rename('abx_days'))
# The eICU docs stress that medication rows are orders, not infusions;
# summing the elapsed minutes between drugstartoffset and drugstopoffset gives a closer DOT estimate.

abx_first  = (med.groupby("patientunitstayid")["drugstartoffset"].min()
                         .rename("onset_abx"))

# ---------- 1-D  Assemble evidence & tier tag -----------------------------
evidence = pd.concat([onset_dx, onset_cx, abx_first, abx_days], axis=1).fillna(0)

def tag(r):
    if r.abx_days >= 4 and r.onset_cx > 0 and r.onset_dx > 0:
        return 5                                           # definite
    if (r.onset_dx > 0 and r.abx_days >= 2) or r.onset_cx > 0:
        return 4                                           # probable
    if (r.abx_days >= 2) or (r.onset_dx > 0):
        return 3                                           # possible
    if (r.abx_days <= 1) and r.onset_dx == 0 and r.onset_cx == 0:
        return 1                                           # definite clean
    return 2                                               # probably clean

evidence["infection_tag"] = evidence.apply(tag, axis=1).astype("int8")

# Earliest onset among any evidence type
evidence["infection_onset"] = evidence[["onset_dx", "onset_cx", "onset_abx"]].\
                                 replace(0, np.nan).min(axis=1)
evidence.reset_index(inplace=True)     # → patientunitstayid column

# Why this mix? Explicit sepsis ICD-9 codes are highly specific but miss cases; 
# adding culture and ≥ 4 days of antibiotics matches the CDC “Adult Sepsis Event” 
# definition for presumed infection and restores sensitivity while still keeping PPV high.

#Print the qty of each tag
print("Infection Tag Counts:")
print(evidence["infection_tag"].value_counts())

# Check if there are any NaN values in the infection_tag column
nan_count = evidence["infection_tag"].isna().sum()
print("NaN count in infection_tag column:", nan_count)
# # How many stays hit each criterion?
# crit = evidence.assign(
#     has_sepsis_code = evidence['onset_dx']>0,
#     pos_culture     = evidence['onset_cx']>0,
#     abx_ge2         = evidence['abx_days']>=2,
#     abx_ge4         = evidence['abx_days']>=4
# )

# summary = crit[['has_sepsis_code','pos_culture','abx_ge2','abx_ge4']].mean()
# print(summary)          # shows prevalence of each rule
# print('all three (4-day rule):',
#       ((crit.has_sepsis_code)&(crit.pos_culture)&(crit.abx_ge4)).sum())

C:\Users\orosh\AppData\Local\Temp\ipykernel_24300\4001151145.py:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  dx["infect_txt"]  = dx["diagnosisstring"].fillna("").str.contains(txt_re)
C:\Users\orosh\AppData\Local\Temp\ipykernel_24300\4001151145.py:45: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  med["is_abx"] = med["drugname"].fillna("").str.contains(abx_pat)


Infection Tag Counts:
infection_tag
3    50493
1    15747
4    15152
2     5917
5       69
Name: count, dtype: int64
NaN count in infection_tag column: 0


In [13]:
#Save the evidence DataFrame to a CSV file
evidence_path = os.path.join(OUTPUT_PATH, "evidence.csv.gz")
evidence.to_csv(evidence_path, index=False, compression="gzip")

In [14]:
# columns to read from vitalPeriodic.csv
vitals_columns_to_use = [
    "patientunitstayid",   # ICU stay identifier            - Necessary
    "observationoffset",   # minutes since unit admission   - Necessary
    "temperature",         # °C                             - Always keep
    "heartrate",           # beats/min                      - Always keep
    "respiration",         # breaths/min                    - Always keep
    "sao2",                # percent SpO2                   - Keep if sensor coverage ≥ 80 %
    "systemicsystolic",    # mmHg                           - Keep (at least MAP or SBP)
    "systemicdiastolic",   # mmHg                           - Keep (at least MAP or SBP)
    "systemicmean",        # mmHg (MAP)                     - Keep (at least MAP or SBP)
    "etco2",               # mmHg                           - Useful but optional (coverage ~10 %)
    "cvp"                  # mmHg                           - Consider only if ≥ 15 % present
]

'''
| Scenario                   | Columns to keep                                                                  | Typical AUROC hit\*                                                                        |
| -------------------------- | -------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------ |
| **Core model (6 vars)**    | temperature, respiration, heartrate, systemicmean (or SBP/DBP pair), sao2, etco2 | < 2 % lower than full set in published eICU/TB-LSTM tests                                  |
| **Slim model (4 vars)**    | temperature, respiration, heartrate, systemicmean                                | \~4 % lower AUROC but still > 0.80 in most tree-based models                               |
| **Ultra-minimal (3 vars)** | temperature, respiration, heartrate                                              | Good for embedded / streaming; catches 70-80 % of cases that meet SIRS in eICU simulations |
'''

base_columns = [
    "patientunitstayid",   # ICU stay identifier            - Necessary
    "observationoffset",   # minutes since unit admission   - Necessary
]

scenarios = {
    "core": [
        "temperature",
        "respiration",
        "heartrate",
        # "systemicsystolic",
        # "systemicdiastolic", 
        "systemicmean",  # MAP
        "sao2",
        # "etco2", # Removed because of low coverage (~3 %)
    ],
    "slim": [
        "temperature",
        "respiration",
        "heartrate",
        "systemicmean",
    ],
    "ultra_minimal": [
        "temperature",
        "respiration",
        "heartrate",
    ],
}

columns_to_keep = base_columns + scenarios[SCENARIO]

# nice, short analytic names
vitals_rename_dict = {
    "patientunitstayid": "stay_id",
    "observationoffset": "offset_min",
    "temperature":       "temp_c",
    "heartrate":         "HR_bpm",
    "respiration":       "RR_bpm",
    "sao2":              "spo2_pct",
    "systemicsystolic":  "SYS_mmHg",
    "systemicdiastolic": "DIA_mmHg",
    "systemicmean":      "MAP_mmHg",
    "etco2":             "etco2_mmHg",
    "cvp":               "cvp_mmHg"
}

# memory-efficient dtypes (32-bit floats; integer IDs)
vitals_dtypes = {
    "patientunitstayid": "int32",
    "observationoffset": "int32",
    "temperature":       "float32",
    "heartrate":         "int32",
    "respiration":       "int32",
    "sao2":              "int32",
    "systemicsystolic":  "int32",
    "systemicdiastolic": "int32",
    "systemicmean":      "int32",
    "etco2":             "int32",
    "cvp":               "int32"
}

# Column: patientunitstayid, Average difference: 0.00, Coverage: 100.00%, Coverage per patient: 100.00%
# Column: observationoffset, Average difference: 0.00, Coverage: 100.00%, Coverage per patient: 100.00%
# Column: temperature, Average difference: 0.25, Coverage: 7.25%, Coverage per patient: 8.11%
# Column: sao2, Average difference: 0.00, Coverage: 90.19%, Coverage per patient: 97.95%
# Column: heartrate, Average difference: 0.00, Coverage: 99.55%, Coverage per patient: 99.66%
# Column: respiration, Average difference: 0.00, Coverage: 85.86%, Coverage per patient: 90.46%
# Column: cvp, Average difference: 0.00, Coverage: 11.78%, Coverage per patient: 13.45%
# Column: etco2, Average difference: 0.00, Coverage: 3.22%, Coverage per patient: 3.96%
# Column: systemicsystolic, Average difference: 0.00, Coverage: 18.59%, Coverage per patient: 23.63%
# Column: systemicdiastolic, Average difference: 0.00, Coverage: 18.59%, Coverage per patient: 23.63%
# Column: systemicmean, Average difference: 0.00, Coverage: 18.73%, Coverage per patient: 23.76%

# reading in chunks because of memory limitations
chunk_size = 1_000_000       # rows
out_dir    = os.path.join(OUTPUT_PATH, "agg_parts")
os.makedirs(out_dir, exist_ok=True)

part = 0                      # incremental file counter
for chunk in tqdm(
        pd.read_csv(
            os.path.join(DATA_PATH, VITALS_DATA_PATH),
            usecols=columns_to_keep,
            chunksize=chunk_size
        ),
        desc="Loading Vitals Data"):

    # --- 1. rename columns for readability --------------------------------
    chunk.rename(columns=vitals_rename_dict, inplace=True)     # stay_id, offset_min, ...

    #DEBUG: Print all column name
    # print("Columns in chunk:", chunk.columns.tolist())

    # --- 2. build a real time index (needed for resample) ------------------
    #print offset min and max
    # print("Offset min:", chunk["offset_min"].min(), "Offset max:", chunk["offset_min"].max())
    chunk['ts'] = pd.to_timedelta(chunk['offset_min'], unit='m')
    chunk = chunk.set_index('ts')

    # --- 3. 30-minute aggregation -----------------------------------------
    #Add these extra columns (per 30-min window)
    '''
        - temp_std30, temp_range30, temp_slope30h1  #   Fast swings and diurnal drift flag occult infection.
        - HR_std30, HR_sdnn30, HR_delta_prev        #   HRV metrics and sudden tachycardia bursts.
        - RR_std30, RR_range30                      #   Tachypnea variability rises before hypoxia.
        - spo2_cv30, spo2_min30                     #   Desaturation variability predicts pulmonary source.
        - MAP_std30, MAP_slope30h1                  #	Micro-hypotensive dips precede shock.
    '''

    agg = {
        'temp_c':   ['mean', 'std', 'max', 'min'],
        'HR_bpm':   ['mean', 'std', 'max'],
        'RR_bpm':   ['mean', 'std', 'max', 'min'],
        'MAP_mmHg': ['mean', 'std'],
        'spo2_pct': ['mean', 'std', 'min'],
    }

    agg_30 = (chunk
            .groupby('stay_id')
            .resample('30min')                 # use 'min' instead of deprecated 'T'
            .agg(agg)
            .drop(columns=['offset_min'], errors='ignore')   # harmless if absent
            )

    # flatten MultiIndex columns → e.g. temp_c_mean
    agg_30.columns = ['_'.join(col) for col in agg_30.columns.to_flat_index()]

    # --- 4. derived window-level dynamics ----------------------------------
    # range & coefficient of variation (CV = std / mean)
    agg_30['temp_range30'] = agg_30['temp_c_max'] - agg_30['temp_c_min']
    agg_30['RR_range30']   = agg_30['RR_bpm_max']  - agg_30['RR_bpm_min']
    agg_30['spo2_cv30']    = agg_30['spo2_pct_std'] / agg_30['spo2_pct_mean']
    agg_30['HR_sdnn30']    = agg_30['HR_bpm_std']          # SDNN proxy

    # slope / delta (per hour; 30-min step = 0.5 h)
    for base in ['temp_c_mean', 'HR_bpm_mean', 'RR_bpm_mean', 'MAP_mmHg_mean']:
        agg_30[f"{base.replace('_mean','')}_slope30h1"] = (
            agg_30.groupby('stay_id')[base].diff() / 0.5
        )
    agg_30['HR_delta_prev'] = agg_30.groupby('stay_id')['HR_bpm_mean'].diff()
    agg_30['MAP_slope30h1'] = agg_30['MAP_mmHg_std'] / agg_30['MAP_mmHg_mean']

    # --- 5. LOCF forward-fill, limit 1 -------------------------------------
    agg_30 = (agg_30
              .groupby('stay_id')
              .ffill(limit=1)            # fill at most the next 30-min slot
              .reset_index()             # bring stay_id & ts back as columns
             )
    
    # restore the original offset_min column
    # agg_30['offset_min'] = offset_min_col
    agg_30['offset_min'] = (agg_30['ts']
                            .dt.total_seconds()
                            .div(60)
                            .astype('int32'))
    
    # print("Aggset min:", agg_30["offset_min"].min(), "Offset max:", agg_30["offset_min"].max())

    # --- 5.1. Keep only the relevant columns -------------------------------
    # Keep only the columns we need
    #Add these extra columns (per 30-min window)
    '''
        - temp_std30, temp_range30, temp_slope30h1  #   Fast swings and diurnal drift flag occult infection.
        - HR_std30, HR_sdnn30, HR_delta_prev        #   HRV metrics and sudden tachycardia bursts.
        - RR_std30, RR_range30                      #   Tachypnea variability rises before hypoxia.
        - spo2_cv30, spo2_min30                     #   Desaturation variability predicts pulmonary source.
        - MAP_std30, MAP_slope30h1                  #	Micro-hypotensive dips precede shock.
    '''

    agg_30 = agg_30[[
        'stay_id', 'ts', 'offset_min',
        'temp_c_mean', 'temp_c_std', 'temp_range30', 'temp_c_slope30h1',
        'HR_bpm_mean', 'HR_bpm_std', 'HR_sdnn30', 'HR_delta_prev',
        'RR_bpm_mean', 'RR_bpm_std', 'RR_range30',
        'MAP_mmHg_mean', 'MAP_mmHg_std', 'MAP_slope30h1',
        'spo2_pct_mean', 'spo2_pct_std', 'spo2_cv30', 'spo2_pct_min'
    ]]

    # --- 6. Merge with infection times ----------------------------------
    # merge the onset table
    agg_30 = agg_30.merge(
                evidence[["patientunitstayid", "infection_tag", "infection_onset"]],
                left_on="stay_id",
                right_on="patientunitstayid",
                how="left"
            )

    # binary 0/1 flag
    agg_30["infected"] = (
            agg_30["infection_onset"].notna() &
            (agg_30["offset_min"] >= agg_30["infection_onset"])
        ).astype("int8")

    # keep only what the time-series file needs
    agg_30.drop(columns=["infection_tag", "infection_onset", "patientunitstayid"],
                inplace=True)

    # --- 7. persist this part to disk --------------------------------------
    part_file = os.path.join(out_dir, f"vitals_30min_part_{part:04d}.parquet")
    agg_30.to_parquet(part_file, index=False)
    part += 1

    # #DEBUG: Stop after 5 chunks
    # if part == 1:
    #     break

Loading Vitals Data: 147it [34:07, 13.93s/it]


In [15]:
#Concatenate all the parts
out_dir = os.path.join(OUTPUT_PATH, "agg_parts")
agg_parts = []
for part in tqdm(
        os.listdir(out_dir),
        desc="Loading Vitals Parts"):
    part_file = os.path.join(out_dir, part)
    agg_parts.append(pd.read_parquet(part_file))
df_vitals = pd.concat(agg_parts, ignore_index=True)
#print head
print(df_vitals.head())
#TODO: Save dataset, static data
#Save the evidence DataFrame to a CSV file
print("Shape of df_vitals:", df_vitals.shape)
print("Individual stay_ids:", df_vitals["stay_id"].nunique())
#count the number of patients with at least one infected row
infected_patients = df_vitals[df_vitals["infected"] == 1]["stay_id"].nunique()
print("Number of infected patients:", infected_patients)
vitals_path = os.path.join(OUTPUT_PATH, "vitals.csv.gz")
df_vitals.to_csv(vitals_path, index=False, compression="gzip")

Loading Vitals Parts: 100%|██████████| 147/147 [00:10<00:00, 14.08it/s]


   stay_id              ts  offset_min  temp_c_mean  temp_c_std  temp_range30  \
0   141168 0 days 01:59:00         119          NaN         NaN           NaN   
1   141168 0 days 02:29:00         149          NaN         NaN           NaN   
2   141168 0 days 02:59:00         179          NaN         NaN           NaN   
3   141168 0 days 03:29:00         209          NaN         NaN           NaN   
4   141168 0 days 03:59:00         239          NaN         NaN           NaN   

   temp_c_slope30h1  HR_bpm_mean  HR_bpm_std  HR_sdnn30  ...  RR_bpm_std  \
0               NaN   140.000000    0.000000   0.000000  ...         NaN   
1               NaN   134.666667    3.011091   3.011091  ...         NaN   
2               NaN   134.333333    0.816497   0.816497  ...         NaN   
3               NaN   134.000000    0.000000   0.000000  ...         NaN   
4               NaN   133.000000    1.095445   1.095445  ...         NaN   

   RR_range30  MAP_mmHg_mean  MAP_mmHg_std  MAP_slope30h

| Keep?                                      | Column(s)                                                                                                                                             | Rationale                                                                                                                                   |
| ------------------------------------------ | ----------------------------------------------------------------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------- |
| **✅ Keep**                                 | `age`                                                                                                                                                 | Infection and sepsis risk rise sharply above 65 y; age is independent of acute physiology in risk models ([PMC][1]).                        |
| **✅ Keep**                                 | `gender`                                                                                                                                              | Large multicentre studies show sex-specific mortality and treatment gaps in septic patients ([PMC][2]).                                     |
| **✅ Keep**                                 | `admissionweight` (or BMI)                                                                                                                            | Obesity associates with higher device-related infection rates and dosing challenges ([PMC][3]).                                             |
| **✅ Keep**                                 | `apacheadmissiondx` (high-level diagnosis *without* using the actual APACHE score)                                                                    | Captures surgical vs. medical cases and baseline organ focus; improves discrimination without leaking future SOFA/APACHE values ([PMC][1]). |
| **✅ Keep (categorical one-hot/embedding)** | `unitadmitsource`, `hospitaladmitsource`, `unittype`                                                                                                  | Infection incidence differs between direct-ED admits, ward transfers and surgical ICUs ([PMC][4]).                                          |
| **🔶 Consider**                            | `ethnicity`                                                                                                                                           | Adds modest uplift in some U.S. cohorts; effect sizes smaller than age/sex.                                                                 |
| **🚫 Drop**                                | IDs (`patientunitstayid`, `patienthealthsystemstayid`, `uniquepid`), discharge-time fields, `hospitaldischarge*`, `unitdischarge*`, `dischargeweight` | Pure identifiers or post-event information → no predictive value or label leakage.                                                          |
| **🚫 Drop**                                | `hospitalid`, `wardid`                                                                                                                                | Site effects are large; unless you explicitly model site-level random effects, they can overfit.                                            |

[1]: https://pmc.ncbi.nlm.nih.gov/articles/PMC3580738/?utm_source=chatgpt.com "Risk assessment in sepsis: a new prognostication rule by APACHE II ..."
[2]: https://pmc.ncbi.nlm.nih.gov/articles/PMC3322379/?utm_source=chatgpt.com "Gender Differences in Mortality in Patients with Severe Sepsis and ..."
[3]: https://pmc.ncbi.nlm.nih.gov/articles/PMC8340548/?utm_source=chatgpt.com "Impact of Obesity in Critical Illness - PMC - PubMed Central"
[4]: https://pmc.ncbi.nlm.nih.gov/articles/PMC3862707/?utm_source=chatgpt.com "Incidence of healthcare associated infection in the surgical ICU of a ..."


In [16]:

# ------------------------------------------------------------------ #
# 1  Columns to load (add unitDischargeOffset so we can drop short stays)
patient_columns_to_use = [
    "patientunitstayid", 
    "age", 
    "gender", 
    "admissionweight",
    "apacheadmissiondx", 
    "unitadmitsource", 
    "hospitaladmitsource",
    "unittype", 
    "unitdischargeoffset"          # LOS in minutes
]

# ------------------------------------------------------------------ #
# 2  Memory-efficient dtypes
patient_dtypes = {
    "patientunitstayid"   : "int32",
    "age"                 : "object",      # string like "76" or "> 89"
    "gender"              : "category",
    "admissionweight"     : "float32",
    "apacheadmissiondx"   : "category",
    "unitadmitsource"     : "category",
    "hospitaladmitsource" : "category",
    "unittype"            : "category",
    "unitdischargeoffset" : "int32",
}

# ------------------------------------------------------------------ #
# 3  Consistent snake-case column names
patient_rename_dict = {
    "patientunitstayid"   : "stay_id",
    "age"                 : "age_str",
    "gender"              : "gender",
    "admissionweight"     : "weight_kg",
    "apacheadmissiondx"   : "admit_dx",
    "unitadmitsource"     : "unit_admit_source",
    "hospitaladmitsource" : "hosp_admit_source",
    "unittype"            : "unit_type",
    "unitdischargeoffset" : "discharge_offset",
}

# ------------------------------------------------------------------ #
# 4  Load PATIENT table
patient_data = pd.read_csv(
    os.path.join(DATA_PATH, PATIENT_DATA_PATH),
    usecols     = patient_columns_to_use,
    dtype       = patient_dtypes,
    low_memory  = False
).rename(columns = patient_rename_dict)

# ------------------------------------------------------------------ #
# 5  Convert age string → numeric and drop minors
patient_data["age"] = (patient_data["age_str"]
                       .str.replace("> ?89", "90", regex=True)  # "> 89" → 90
                       .astype("float32"))
patient_data = patient_data[patient_data["age"] >= 18]
# Drop the age_str column
patient_data.drop(columns=["age_str"], inplace=True)

# ------------------------------------------------------------------ #
# 6  Drop very short ICU stays (≤ 60 min LOS)
MIN_LOS_MIN = 60
patient_data = patient_data[patient_data["discharge_offset"] >= MIN_LOS_MIN]

# ------------------------------------------------------------------ #
# 7  Attach the infection evidence tag (one row per stay_id)
static_df = patient_data.merge(
    evidence[["patientunitstayid", "infection_tag"]]
            .rename(columns={"patientunitstayid": "stay_id"}),
    on   = "stay_id",
    how  = "left",
    validate = "one_to_one"
).reset_index(drop=True)

# ------------------------------------------------------------------ #
# static_df now has exactly one row per stay_id with:
# stay_id | age (float) | gender | weight_kg | admit_dx | unit_admit_source
# hosp_admit_source | unit_type | discharge_offset | infection_tag
# Save once for downstream modelling
# static_df.to_parquet(
#     os.path.join(DATA_PATH, "static_patient_info.parquet"),
#     engine="pyarrow",
#     index=False
# )
# Drop if infection tag is NaN
# static_df = static_df.dropna(subset=["infection_tag"])
# Fill infection_tag with 0 if NaN
static_df["infection_tag"] = static_df["infection_tag"].fillna(0).astype("int8")

static_df.to_csv(
    os.path.join(OUTPUT_PATH, "static_patient_info.csv.gz"),
    index=False,
    compression="gzip"
)

# Print the first 40 rows of the static_df DataFrame
print("Static Patient DataFrame:")
print(static_df.head(40))
# Print total number of rows
print("Total number of rows in static_df:", len(static_df))

Static Patient DataFrame:
    stay_id  gender                                           admit_dx  \
0    141168  Female      Rhythm disturbance (atrial, supraventricular)   
1    141179  Female                                                NaN   
2    141194    Male              Sepsis, renal/UTI (including bladder)   
3    141196    Male                                                NaN   
4    141197    Male                                  Sepsis, pulmonary   
5    141203  Female       Arrest, respiratory (without cardiac arrest)   
6    141208  Female  Overdose, sedatives, hypnotics, antipsychotics...   
7    141227    Male                                  Sepsis, pulmonary   
8    141229  Female                      CHF, congestive heart failure   
9    141233  Female                           Mitral valve replacement   
10   141244    Male                    Graft, femoral-popliteal bypass   
11   141260  Female                                             Asthma   
12   141263 

In [17]:
# #Compare size of processed vitals and original vitals to see if it makes sense
# chunk_size = 1_000_000       # rows
# n_rows = 0
# n_cols = 0
# # Load the original vitals data
# # for chunk in tqdm(
# #         pd.read_csv(
# #             os.path.join(DATA_PATH, VITALS_DATA_PATH),
# #             usecols=columns_to_keep,
# #             chunksize=chunk_size
# #         ),
# #         desc="Loading Original Vitals Data"):
# #     n_rows += chunk.shape[0]
# #     n_cols = chunk.shape[1]
# # Print the shape of the original vitals data
# print("Original Vitals Data Shape:", (n_rows, n_cols))
# # Print the shape of the processed vitals data

# #Load processed vitals data
# df_vitals = pd.read_csv(vitals_path)

# print("Processed Vitals Data Shape:", df_vitals.shape)
# # 146,671,642
# # 25,174,325

# # 146,671,642 / 30 * 5 = 24,445,273
# # 25,174,325 * 30 / 5 = 151,045,950 => matches the original data

# REPROCESS
Clean vitals data into clean section of 24 hours for each patients, with first 12 hours for label and next 12 to predict, with (if infected) an infection starting around 6h of the prediciton
Keep only patients with enough vitals coverage

In [18]:
#Load vitals data
vitals_path = os.path.join(OUTPUT_PATH, "vitals.csv.gz")
df_vitals = pd.read_csv(vitals_path)
# Count how many patients have at least one infection and how many have none
infected_patients = df_vitals[df_vitals["infected"] == 1]["stay_id"].nunique()
non_infected_patients = df_vitals[df_vitals["infected"] == 0]["stay_id"].nunique()
# Print the number of infected and non-infected patients
print("Number of infected patients:", infected_patients)
print("Number of non-infected patients:", non_infected_patients)
# Count how many infected and non infected patients have at least one of each vitals at any point
# Count how many patients have at least one of each vitals at any point
# Group by stay_id and count the number of unique values in each column
empty = {
    "infected": 0,
    "non_infected": 0
}
complete = {
    "infected": 0,
    "non_infected": 0
}
for stay, group in df_vitals.groupby("stay_id"):
    # Explore each column, if one is entirely empty
    for col in df_vitals.columns:
        if group[col].isna().all():
            # If the column is empty, check if the patient is infected or not
            if group["infected"].iloc[0] == 1:
                empty["infected"] += 1
            else:
                empty["non_infected"] += 1
            break

    if group["infected"].iloc[0] == 1:
        complete["infected"] += 1
    else:
        complete["non_infected"] += 1

print("Number of patients with at least one empty vitals:", empty)
print("Number of patients with at least one complete vitals:", complete)

# Number of infected patients: 81099
# Number of non-infected patients: 159437
# Number of patients with at least one empty vitals: {'infected': 30952, 'non_infected': 151315}
# Number of patients with at least one complete vitals: {'infected': 33401, 'non_infected': 159430}

Number of infected patients: 81099
Number of non-infected patients: 159437
Number of patients with at least one empty vitals: {'infected': 30952, 'non_infected': 151315}
Number of patients with at least one complete vitals: {'infected': 33401, 'non_infected': 159430}


In [19]:
#How many infected patients have at least 12 hours before the onset of infection
infected_patients_stay_id = df_vitals[df_vitals["infected"] == 1]["stay_id"].unique()
infected_patients = 0
infected_patients_12h = 0
for stay in infected_patients_stay_id:
    group = df_vitals[df_vitals["stay_id"] == stay]
    group_start_time = group["offset_min"].min()
    group_infection_time = group[group["infected"] == 1]["offset_min"].min()
    time_diff = group_infection_time - group_start_time
    if time_diff >= 12*60:
        infected_patients_12h += 1
    infected_patients += 1
print("Number of infected patients:", infected_patients)
print("Number of infected patients with at least 12 hours before the onset of infection:", infected_patients_12h)

Number of infected patients: 81099
Number of infected patients with at least 12 hours before the onset of infection: 11635


In [21]:
import pandas as pd, numpy as np, os, math
from tqdm import tqdm

# ------------------------------------------------------------------ #
# 0  paths & constants
DATA_DIR    = r"D:\Users\orosh\Documents\Research\Datasets\eicu-2.0\processed"
VITALS_FILE = os.path.join(DATA_DIR, "vitals.csv.gz")  # 30-min grid, first pass
OUT_DIR     = os.path.join(DATA_DIR, "24h_windows")
os.makedirs(OUT_DIR, exist_ok=True)

WIN_H, HIST_H, BIN_MIN  = 24, 12, 30
BINS_WIN                = WIN_H*60//BIN_MIN    # 48 rows
MIN_DATA_BIN            = 18*60//BIN_MIN       # ≥ 18 h usable rows
FILL_SMALL, FILL_MED    = 2, 6                 # gap sizes (bins)

# ------------------------------------------------------------------ #
dtype = {'stay_id':'int32', 'offset_min':'int32', 'infected':'int8'}
print("Reading first-pass file …")
df = pd.read_csv(VITALS_FILE, dtype=dtype, low_memory=False)

# ------------------------------------------------------------------ #
def dense_enough(row) -> bool:
    """Row counts as data if at least ONE vital is not NaN."""
    return row.drop(['stay_id','ts','offset_min','infected']).notna().any()

def gap_fill(win: pd.DataFrame) -> pd.DataFrame:
    for col in win.columns.difference(['stay_id','ts','offset_min','infected']):
        s = win[col]
        win[col] = (s.interpolate(limit=FILL_SMALL, limit_direction='both')
                       .ffill(limit=FILL_MED)
                       .bfill(limit=FILL_MED))
    return win

def pick_start(stay: pd.DataFrame) -> int | None:
    onset = stay.offset_min[stay.infected==1].min()  # NaN if clean
    step  = BIN_MIN

    if math.isnan(onset):                            # non-infected
        best_left, best_bins = None, 0
        for left in range(stay.offset_min.min(),
                          stay.offset_min.max()-WIN_H*60+step,
                          step):
            ok = stay.loc[(stay.offset_min>=left)&
                           (stay.offset_min<left+WIN_H*60)]\
                     .apply(dense_enough, axis=1).sum()
            if ok > best_bins:
                best_left, best_bins = left, ok
        return best_left if best_bins >= MIN_DATA_BIN else None

    # ---------- infected stays ----------
    for delta in range(6, -1, -1):                  # onset at +6 … +0 h
        left = onset - (HIST_H + delta)*60
        if left < 0:
            continue
        win = stay[(stay.offset_min>=left)&(stay.offset_min<left+WIN_H*60)]
        clean_hist = (win.loc[win.offset_min < left+HIST_H*60,'infected'].sum()==0)
        enough     = win.apply(dense_enough, axis=1).sum() >= MIN_DATA_BIN
        if clean_hist and enough:
            return left
    # fallback: onset @ prediction t=0
    left = onset - HIST_H*60
    win  = stay[(stay.offset_min>=left)&(stay.offset_min<left+WIN_H*60)]
    return left if left>=0 and win.apply(dense_enough,axis=1).sum()>=MIN_DATA_BIN else None

# ------------------------------------------------------------------ #
counts = dict(
    infect_total=0, noninfect_total=0,
    keep_infect=0, keep_noninfect=0,
    sparse=0, no_clean_lead=0, short_los=0
)
samples = []

print("Building 24-h windows …")
for sid, stay in tqdm(df.groupby('stay_id'), total=df['stay_id'].nunique()):
    infected_here = bool(stay['infected'].any())
    counts['infect_total' if infected_here else 'noninfect_total'] += 1

    if stay.apply(dense_enough, axis=1).sum() < MIN_DATA_BIN:
        counts['sparse'] += 1
        continue

    start = pick_start(stay)
    if start is None:
        counts['no_clean_lead' if infected_here else 'sparse'] += 1
        continue

    win = stay[(stay.offset_min>=start)&(stay.offset_min<start+WIN_H*60)].copy()

    # left-pad zeros if window does not reach 24 h
    lag_bins = (win.offset_min.min()-start)//BIN_MIN
    if lag_bins>0:
        pad = pd.DataFrame({
            'stay_id'   : sid,
            'ts'        : pd.NaT,
            'offset_min': np.arange(start, win.offset_min.min(), BIN_MIN, dtype='int32'),
            'infected'  : 0})
        for v in win.columns.difference(pad.columns):
            pad[v] = 0
        win = pd.concat([pad, win], ignore_index=True, sort=False)

    win.sort_values('offset_min', inplace=True)
    win = gap_fill(win)
    # any still-empty rows → zero
    empties = ~win.apply(dense_enough, axis=1)
    win.loc[empties, win.columns.difference(['stay_id','ts','offset_min','infected'])] = 0

    if len(win) != BINS_WIN:
        counts['short_los'] += 1
        continue

    samples.append(win)
    counts['keep_infect' if infected_here else 'keep_noninfect'] += 1

# ------------------------------------------------------------------ #
print("Saving windows …")
pd.concat(samples, ignore_index=True).to_parquet(
    os.path.join(OUT_DIR, "vitals_24h_windows.parquet"),
    engine="pyarrow", index=False
)

# ------------------------------------------------------------------ #
#  summary
print("\n==============  SUMMARY  ==============")
print(f"Infected stays   : {counts['infect_total']}")
print(f"  windows kept   : {counts['keep_infect']}  "
      f"({counts['keep_infect']/max(1,counts['infect_total']):.1%})")
print(f"Non-infected stays : {counts['noninfect_total']}")
print(f"  windows kept     : {counts['keep_noninfect']}  "
      f"({counts['keep_noninfect']/max(1,counts['noninfect_total']):.1%})")
print("Rejections:")
for k in ('sparse','no_clean_lead','short_los'):
    print(f"  {k:<13}: {counts[k]}")
print("========================================")

Reading first-pass file …
Building 24-h windows …


 11%|█▏        | 21959/192831 [1:08:03<8:49:37,  5.38it/s]  


KeyboardInterrupt: 

In [12]:
import pandas as pd, numpy as np, os

# ── file locations ───────────────────────────────────────────────────────
# DATA_DIR    = r"D:\Users\orosh\Documents\Research\Datasets\eicu-2.0\processed"
DATA_DIR     = OUTPUT_PATH
VITALS_CSV   = os.path.join(DATA_DIR, "24h_windows", "vitals_24h_windows.csv.gz")
STATIC_CSV   = os.path.join(DATA_DIR, "static_patient_info.csv.gz")
OUT_CSV      = os.path.join(DATA_DIR, "balanced_24h_windows.csv.gz")

# ── 1. load 24-h windows & infection-evidence tag ────────────────────────
#   • use read_csv for .csv.gz
#   • restrict dtypes to keep memory reasonable
vitals_df = pd.read_csv(
    VITALS_CSV,
    dtype={"stay_id": "int32", "infected": "int8"},   # adjust as needed
    low_memory=False
)                                                    # 48×N rows per stay

static_df = pd.read_csv(
    STATIC_CSV,
    usecols=["stay_id", "infection_tag"],
    dtype={"stay_id": "int32", "infection_tag": "int8"}
)

In [68]:
vitals = vitals_df.copy()
static = static_df.copy()

#print how many rows in vitals and how many unique stay_ids that is
#print how many in vitals are infected and how many are not
print("Number of rows in vitals:", len(vitals))
print("Number of unique stay_ids in vitals:", vitals["stay_id"].nunique())
#infected = at least one row with infected == 1 per stay_id
infected = vitals.groupby("stay_id")["infected"].max()
print("Number of infected patients:", infected.sum())
print("Number of non-infected patients:", len(infected) - infected.sum())

# return

# ── 2. mark windows that contain ≥1 infected bin ─────────────────────────
has_inf = (vitals.groupby("stay_id")["infected"]
                   .max()
                   .rename("window_has_infection")
                   .reset_index())

vitals = vitals.merge(has_inf, on="stay_id", how="left")
vitals = vitals.merge(static,  on="stay_id", how="left")

#make infected_tag 0 become 2.5 because we want to prioritize 1 and 2 first
vitals["infection_tag"] = vitals["infection_tag"].replace(0, 2.5)

#print how many infected has temp how many do not, how many not infected have temp and how many do not
has_temp = (vitals.groupby("stay_id")["temp_c_mean"]
                   .max()
                     .rename("window_has_temp")
                     .reset_index())
vitals = vitals.merge(has_temp, on="stay_id", how="left")

temps_count = {}
temps_count["infected_hastemp"] = ((vitals["window_has_infection"] == 1) & (vitals["window_has_temp"] == 1)).sum()
temps_count["infected_notemp"] = ((vitals["window_has_infection"] == 1) & (vitals["window_has_temp"] == 0)).sum()
temps_count["noninfected_hastemp"] = ((vitals["window_has_infection"] == 0) & (vitals["window_has_temp"] == 1)).sum()
temps_count["noninfected_notemp"] = ((vitals["window_has_infection"] == 0) & (vitals["window_has_temp"] == 0)).sum()

print("Number of infected patients with temp:", temps_count["infected_hastemp"])
print("Number of infected patients without temp:", temps_count["infected_notemp"])
print("Number of non-infected patients with temp:", temps_count["noninfected_hastemp"])
print("Number of non-infected patients without temp:", temps_count["noninfected_notemp"])

# count how many rows in the dataset have temp mean not nan
print("Number of rows with temp mean not nan:", vitals["temp_c_mean"].notna().sum())

temp_per_stay_id = {}
#count how many rows in the dataset have temp mean not nan, grouped by stay_id
for stay_id, group in vitals.groupby("stay_id"):
    # if the group has at least one row with temp mean not nan, add to the dict
    if group["temp_c_mean"].notna().any():
        temp_per_stay_id[stay_id] = group["temp_c_mean"].notna().sum()
    else:
        temp_per_stay_id[stay_id] = 0

# print how many rows in the dataset have temp mean not nan, grouped by stay_id
print("Number of rows with temp mean not nan, grouped by stay_id:")

#count how many are = 0 in the dict
temp_count = 0
for stay_id, count in temp_per_stay_id.items():
    if count == 0:
        temp_count += 1

print("Number of stays with temp mean nan:", temp_count)
print("Number of stays with temp mean not nan", len(temp_per_stay_id) - temp_count)

# -- Print a table that shows the number of infected and non-infected patients grouped by stay id by infection tag
# infected = 1, non-infected = 0
# infection_tag = 0, 1, 2, 3, 4
# one_per_stay = (vitals
#                 .groupby(["stay_id", "window_has_infection", "infection_tag"])
#                 .size()
#                 .unstack(fill_value=0)
#                 .rename_axis(index=None, columns="infection_tag")
#                 .reset_index()
#                 )

grouped = (vitals
            .groupby(["window_has_infection", "infection_tag"])
            .size()
            .unstack(fill_value=0)
            .rename_axis(index=None, columns="infection_tag")
            .reset_index()
            )
grouped = grouped / 48
display(grouped)

# ── 3. counts & target sample size ───────────────────────────────────────
n_infected = has_inf["window_has_infection"].sum()
print(f"Total infected stays in windows : {n_infected}")

# ── 4. choose the same number of clean stays, prioritising temp coverage then low tag ───────

# 4-a. count how many non-NaN temperature rows each stay has
temp_counts = (vitals
               .groupby("stay_id")["temp_c_mean"]
               .apply(lambda s: s.notna().sum())
               .rename("num_temp_rows")
               .reset_index())

# 4-b. build the clean-stay pool, ordered by:
#      1) most temperature measurements (descending)
#      2) lowest infection_tag (ascending)
clean_pool = (
    vitals.query("window_has_infection == 0")[["stay_id", "infection_tag"]]
          .drop_duplicates()
          .merge(temp_counts, on="stay_id", how="left")
          .sort_values(["num_temp_rows", "infection_tag"],
                       ascending=[False, True])
)

# 4-c. take as many clean stays as there are infected ones
chosen_clean_ids = clean_pool.head(int(n_infected))["stay_id"]

# (rest of the pipeline remains unchanged)
infected_ids = vitals.query("window_has_infection == 1")["stay_id"].unique()
final_ids    = np.concatenate([infected_ids, chosen_clean_ids])

balanced = vitals[vitals["stay_id"].isin(final_ids)].copy()

# # ── 4. choose the same number of clean stays, prioritising low tag ───────


# clean_pool = (
#     vitals.query("window_has_infection == 0")[["stay_id", "infection_tag"]]
#           .drop_duplicates()
#           .sort_values("infection_tag")            # best evidence (1) first
# )

# chosen_clean_ids = clean_pool.head(int(n_infected))["stay_id"]

# infected_ids = vitals.query("window_has_infection == 1")["stay_id"].unique()
# final_ids    = np.concatenate([infected_ids, chosen_clean_ids])

# balanced = vitals[vitals["stay_id"].isin(final_ids)].copy()

# ── 5. save and report ───────────────────────────────────────────────────
balanced.to_csv(OUT_CSV, index=False, compression="gzip")

summary = (balanced.drop_duplicates("stay_id")
                     .groupby(["window_has_infection", "infection_tag"])
                     .size()
                     .unstack(fill_value=0))

print(f"\nSaved balanced set → {OUT_CSV}")
print("\nPatients kept (rows = infected 0/1, columns = evidence tag):")
display(summary)

Number of rows in vitals: 3533568
Number of unique stay_ids in vitals: 73616
Number of infected patients: 10311
Number of non-infected patients: 63305
Number of infected patients with temp: 0
Number of infected patients without temp: 21216
Number of non-infected patients with temp: 0
Number of non-infected patients without temp: 1920
Number of rows with temp mean not nan: 324179
Number of rows with temp mean not nan, grouped by stay_id:
Number of stays with temp mean nan: 65619
Number of stays with temp mean not nan 7997


infection_tag,index,1.0,2.0,2.5,3.0,4.0,5.0
0,0.000000,823.0,327.0,60973.0,948.0,1.0,0.0
1,0.020833,1941.0,912.0,0.0,6650.0,786.0,4.0


Total infected stays in windows : 10311

Saved balanced set → D:\Users\orosh\Documents\Research\Datasets\eicu-2.0\processed\balanced_24h_windows.csv.gz

Patients kept (rows = infected 0/1, columns = evidence tag):


infection_tag,1.0,2.0,2.5,3.0,4.0,5.0
window_has_infection,,,,,,
0,823,327,9087,50,0,0
1,1941,912,0,6650,786,4


In [127]:
# print a random window of 10 rows from the balanced DataFrame

#pick a random pos
random_window_start = np.random.randint(0, len(balanced)-10)
random_window_end = random_window_start + 10

random_window = balanced.iloc[random_window_start:random_window_end]
print("Random window of 10 rows from the balanced DataFrame:")
random_window.head(10)

Random window of 10 rows from the balanced DataFrame:


,stay_id,ts,offset_min,temp_c_mean,temp_c_std,temp_range30,temp_c_slope30h1,HR_bpm_mean,HR_bpm_std,HR_sdnn30,...,MAP_mmHg_std,MAP_slope30h1,spo2_pct_mean,spo2_pct_std,spo2_cv30,spo2_pct_min,infected,window_has_infection,infection_tag,window_has_temp
2034234,1853966,5 days 06:35:00,7595,NaN,NaN,NaN,NaN,79.600000,3.286335,3.286335,...,NaN,NaN,100.000000,0.000000,0.000000,100.0,1,1,3.0,NaN
2034235,1853966,5 days 07:05:00,7625,NaN,NaN,NaN,NaN,83.333333,8.358628,8.358628,...,NaN,NaN,100.000000,0.000000,0.000000,100.0,1,1,3.0,NaN
2034236,1853966,5 days 07:35:00,7655,NaN,NaN,NaN,NaN,103.333333,6.531973,6.531973,...,NaN,NaN,100.000000,0.000000,0.000000,100.0,1,1,3.0,NaN
2034237,1853966,5 days 08:05:00,7685,NaN,NaN,NaN,NaN,100.333333,12.484657,12.484657,...,NaN,NaN,100.000000,0.000000,0.000000,100.0,1,1,3.0,NaN
2034238,1853966,5 days 08:35:00,7715,NaN,NaN,NaN,NaN,86.000000,0.000000,0.000000,...,NaN,NaN,100.000000,0.000000,0.000000,100.0,1,1,3.0,NaN
2034239,1853966,5 days 09:05:00,7745,NaN,NaN,NaN,NaN,86.666667,2.422120,2.422120,...,NaN,NaN,100.000000,0.000000,0.000000,100.0,1,1,3.0,NaN
2034240,1853981,0 days 02:32:00,152,NaN,NaN,NaN,NaN,86.333333,1.366260,1.366260,...,NaN,NaN,100.000000,0.000000,0.000000,100.0,0,1,4.0,NaN
2034241,1853981,0 days 03:02:00,182,NaN,NaN,NaN,NaN,86.833333,0.752773,0.752773,...,NaN,NaN,100.000000,0.000000,0.000000,100.0,0,1,4.0,NaN
2034242,1853981,0 days 03:32:00,212,NaN,NaN,NaN,NaN,90.000000,3.949684,3.949684,...,NaN,NaN,99.833333,0.408248,0.004089,99.0,0,1,4.0,NaN
2034243,1853981,0 days 04:02:00,242,NaN,NaN,NaN,NaN,89.166667,1.940790,1.940790,...,NaN,NaN,100.000000,0.000000,0.000000,100.0,0,1,4.0,NaN


In [128]:
#print all cols names in static_patient_info.csv.gz and balanced_24h_windows.csv.gz
static_cols = pd.read_csv(STATIC_CSV).columns.tolist()
print("Columns in static_patient_info.csv.gz:")
print(static_cols)
# print all cols names in balanced_24h_windows.csv.gz
balanced_cols = pd.read_csv(OUT_CSV).columns.tolist()
print("Columns in balanced_24h_windows.csv.gz:")
print(balanced_cols)

# Columns in static_patient_info.csv.gz:
# ['stay_id', 'gender', 'admit_dx', 'hosp_admit_source', 'unit_type', 'unit_admit_source', 'weight_kg', 'discharge_offset', 'age', 'infection_tag']
# Columns in balanced_24h_windows.csv.gz:
# ['stay_id', 'ts', 'offset_min', 'temp_c_mean', 'temp_c_std', 'temp_range30', 'temp_c_slope30h1', 'HR_bpm_mean', 'HR_bpm_std', 'HR_sdnn30', 'HR_delta_prev', 'RR_bpm_mean', 'RR_bpm_std', 'RR_range30', 'MAP_mmHg_mean', 'MAP_mmHg_std', 'MAP_slope30h1', 'spo2_pct_mean', 'spo2_pct_std', 'spo2_cv30', 'spo2_pct_min', 'infected', 'window_has_infection', 'infection_tag', 'window_has_temp']

Columns in static_patient_info.csv.gz:
['stay_id', 'gender', 'admit_dx', 'hosp_admit_source', 'unit_type', 'unit_admit_source', 'weight_kg', 'discharge_offset', 'age', 'infection_tag']
Columns in balanced_24h_windows.csv.gz:
['stay_id', 'ts', 'offset_min', 'temp_c_mean', 'temp_c_std', 'temp_range30', 'temp_c_slope30h1', 'HR_bpm_mean', 'HR_bpm_std', 'HR_sdnn30', 'HR_delta_prev', 'RR_bpm_mean', 'RR_bpm_std', 'RR_range30', 'MAP_mmHg_mean', 'MAP_mmHg_std', 'MAP_slope30h1', 'spo2_pct_mean', 'spo2_pct_std', 'spo2_cv30', 'spo2_pct_min', 'infected', 'window_has_infection', 'infection_tag', 'window_has_temp']


In [148]:
#pick a random patient in vitals, say how many rows it has
static = pd.read_csv(STATIC_CSV)
vitals = pd.read_csv(OUT_CSV)
random_patient = np.random.randint(0, len(vitals)-1)
random_patient_id = vitals.iloc[random_patient]["stay_id"]
random_patient_rows = vitals[vitals["stay_id"] == random_patient_id]
print("Random patient ID:", random_patient_id)
print("Number of rows for this patient:", len(random_patient_rows))

Random patient ID: 246400
Number of rows for this patient: 48


In [149]:
#print shape of both dataset
print("Shape of vitals dataset:", vitals.shape)
print("Shape of static dataset:", static.shape)

Shape of vitals dataset: (989856, 25)
Shape of static dataset: (195339, 10)
